# InPost - parcel volume forecasting

## Dataset overview

The data contains 4 datasets with different data between January 2021 and Sep 2023:

- dimDates.csv - data with the informations of the days (year, month, day of the week, isHoliday, etc.)
- Posting_volumes.parquet - data with parcel volumes from Jan 2021 to Sep 2023
- X_ClientORDERS.xlsx - 

In [252]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df1 = pd.read_csv("data/dimDates.csv", sep = ";")
df1

,datePk,dateId,dateYear,dateYearISO,dateMonthYear,dateWeekYearISO,dateQuarterYear,dateMonth,dateDay,dateDaySuffix,...,courierBillingPeriodStartDateFk,courierBillingPeriodEndDateFk,xlsDate,dwhDateFrom,dwhDateTo,dwhCurrRec,dwhAuditCd,dwhAuditCp,dwhAuditMd,dwhAuditMp
0,20201004,2020-10-04,2020,2020,10.2020,T40-2020,Q4-2020,10,4,th,...,20201001,20201015,44108,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
1,20200810,2020-08-10,2020,2020,8.2020,T33-2020,Q3-2020,8,10,th,...,20200801,20200815,44053,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
2,20200930,2020-09-30,2020,2020,9.2020,T40-2020,Q3-2020,9,30,th,...,20200916,20200930,44104,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
3,20200521,2020-05-21,2020,2020,5.2020,T21-2020,Q2-2020,5,21,st,...,20200516,20200531,43972,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
4,20200527,2020-05-27,2020,2020,5.2020,T22-2020,Q2-2020,5,27,th,...,20200516,20200531,43978,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,20200305,2020-03-05,2020,2020,3.2020,T10-2020,Q1-2020,3,5,th,...,20200301,20200315,43895,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
1457,20200417,2020-04-17,2020,2020,4.2020,T16-2020,Q2-2020,4,17,th,...,20200416,20200430,43938,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
1458,20201104,2020-11-04,2020,2020,11.2020,T45-2020,Q4-2020,11,4,th,...,20201101,20201115,44139,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
1459,20201116,2020-11-16,2020,2020,11.2020,T47-2020,Q4-2020,11,16,th,...,20201116,20201130,44151,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719


In [254]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 53 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   datePk                                               1461 non-null   int64  
 1   dateId                                               1461 non-null   object 
 2   dateYear                                             1461 non-null   int64  
 3   dateYearISO                                          1461 non-null   int64  
 4   dateMonthYear                                        1461 non-null   float64
 5   dateWeekYearISO                                      1461 non-null   object 
 6   dateQuarterYear                                      1461 non-null   object 
 7   dateMonth                                            1461 non-null   int64  
 8   dateDay                                              1461 non-null  

In [255]:
df1.dateMonthYear.unique()

array([10.202 ,  8.202 ,  9.202 ,  5.202 ,  6.202 ,  2.2021, 11.202 ,
        5.2021,  8.2021,  3.2023,  4.2023,  8.2023,  9.2023, 10.2023,
       11.2023,  3.202 ,  2.2023,  5.2023,  1.202 , 12.2023,  9.2022,
       10.2022,  1.2022,  5.2022,  4.2022,  8.2022,  6.2023, 12.2022,
       11.2022,  7.2022,  9.2021, 10.2021,  7.2023, 12.202 ,  4.202 ,
       11.2021,  4.2021,  3.2022,  2.2022,  6.2021,  7.2021,  1.2023,
       12.2021,  2.202 ,  1.2021,  7.202 ,  6.2022,  3.2021])

In [256]:
df1.dateId = pd.to_datetime(df1.dateId)
df1 = df1[(df1.dateId >= "2021-01-01") & (df1.dateId <= "2023-08-31")]
df1 = df1.rename(columns = {"dateId": "date"})
df1 = df1.sort_values(by = "date").reset_index(drop = True)

In [257]:
print("NA:", df1.isna().sum().sum())
print("Duplicated:", df1.duplicated().sum())

NA: 0
Duplicated: 0


In [210]:
df1

,datePk,date,dateYear,dateYearISO,dateMonthYear,dateWeekYearISO,dateQuarterYear,dateMonth,dateDay,dateDaySuffix,...,courierBillingPeriodStartDateFk,courierBillingPeriodEndDateFk,xlsDate,dwhDateFrom,dwhDateTo,dwhCurrRec,dwhAuditCd,dwhAuditCp,dwhAuditMd,dwhAuditMp
0,20210101,2021-01-01,2021,2020,1.2021,T53-2020,Q1-2021,1,1,st,...,20210101,20210115,44197,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
1,20210102,2021-01-02,2021,2020,1.2021,T53-2020,Q1-2021,1,2,nd,...,20210101,20210115,44198,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
2,20210103,2021-01-03,2021,2020,1.2021,T53-2020,Q1-2021,1,3,rd,...,20210101,20210115,44199,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
3,20210104,2021-01-04,2021,2021,1.2021,T1-2021,Q1-2021,1,4,th,...,20210101,20210115,44200,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
4,20210105,2021-01-05,2021,2021,1.2021,T1-2021,Q1-2021,1,5,th,...,20210101,20210115,44201,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,20230827,2023-08-27,2023,2023,8.2023,T34-2023,Q3-2023,8,27,th,...,20230816,20230831,45165,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
969,20230828,2023-08-28,2023,2023,8.2023,T35-2023,Q3-2023,8,28,th,...,20230816,20230831,45166,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
970,20230829,2023-08-29,2023,2023,8.2023,T35-2023,Q3-2023,8,29,th,...,20230816,20230831,45167,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719
971,20230830,2023-08-30,2023,2023,8.2023,T35-2023,Q3-2023,8,30,th,...,20230816,20230831,45168,1900-01-01,9999-12-31,1,2020-07-03 13:51:15.673,1904719,2020-07-03 13:51:15.673,1904719


In [211]:
zero_variance_cols = [col for col in df1.columns if df1[col].nunique() == 1]
zero_variance_cols

['dwhDateFrom',
 'dwhDateTo',
 'dwhCurrRec',
 'dwhAuditCd',
 'dwhAuditCp',
 'dwhAuditMd',
 'dwhAuditMp']

In [212]:
df1 = df1.drop(zero_variance_cols, axis=1)

In [213]:
df1.iloc[:10, 0:20]

,datePk,date,dateYear,dateYearISO,dateMonthYear,dateWeekYearISO,dateQuarterYear,dateMonth,dateDay,dateDaySuffix,dateWeekDay,dateWeekDayName,dateWeekDayNameShort,dateWeekDayNameFirstLetter,dateDayOfYear,dateWeekOfMonth,dateWeekOfYear,dateWeekOfYearISO,dateMonthName,dateMonthNameShort
0,20210101,2021-01-01,2021,2020,1.2021,T53-2020,Q1-2021,1,1,st,6,Friday,FRI,F,1,1,1,53,January,JAN
1,20210102,2021-01-02,2021,2020,1.2021,T53-2020,Q1-2021,1,2,nd,7,Saturday,SAT,S,2,1,1,53,January,JAN
2,20210103,2021-01-03,2021,2020,1.2021,T53-2020,Q1-2021,1,3,rd,1,Sunday,SUN,S,3,2,2,53,January,JAN
3,20210104,2021-01-04,2021,2021,1.2021,T1-2021,Q1-2021,1,4,th,2,Monday,MON,M,4,2,2,1,January,JAN
4,20210105,2021-01-05,2021,2021,1.2021,T1-2021,Q1-2021,1,5,th,3,Tuesday,TUE,T,5,2,2,1,January,JAN
5,20210106,2021-01-06,2021,2021,1.2021,T1-2021,Q1-2021,1,6,th,4,Wednesday,WED,W,6,2,2,1,January,JAN
6,20210107,2021-01-07,2021,2021,1.2021,T1-2021,Q1-2021,1,7,th,5,Thursday,THU,T,7,2,2,1,January,JAN
7,20210108,2021-01-08,2021,2021,1.2021,T1-2021,Q1-2021,1,8,th,6,Friday,FRI,F,8,2,2,1,January,JAN
8,20210109,2021-01-09,2021,2021,1.2021,T1-2021,Q1-2021,1,9,th,7,Saturday,SAT,S,9,2,2,1,January,JAN
9,20210110,2021-01-10,2021,2021,1.2021,T1-2021,Q1-2021,1,10,th,1,Sunday,SUN,S,10,3,3,1,January,JAN


In [214]:
len(df1)-(df1.firstDayNoHolidayNoSaturdayAfter == df1.firstDayNoHolidayNoSaturdayAfterData).sum()

1

In [215]:
(df1[df1.firstDayNoHolidayNoSaturdayAfter != df1.firstDayNoHolidayNoSaturdayAfterData]).loc[:, 
                                    ["date", "firstDayNoHolidayNoSundayAfter", "firstDayNoHolidayNoSaturdayAfterData"]]

,date,firstDayNoHolidayNoSundayAfter,firstDayNoHolidayNoSaturdayAfterData
356,2021-12-23,20211227,20201227


In [216]:
df1 = df1.drop(["dateDaySuffix", "dateWeekDayName", "dateWeekDayNameShort", "dateWeekDayNameFirstLetter", 
                "dateMonthName", "dateMonthNameShort", "dateMonthNameFirstLetter", "dateQuarterYear", "dateMonthYear",
                "dateQuarterShort", "dateWeekDayNamePL", "dateMonthNamePL", "xlsDate", "datePk", "dateYearISO",
                "dateQuarterName", "dateWeekYearISO", "dateWeekDay", "dateWeekOfYear", "dateWeekOfYearISO",
                "firstDayNoHolidayNoSaturdayAfterData", "firstDayNoSaturdayNoHolidayOrSundayHolidayName", "courierBillingPeriodDateFk"], axis = 1)

In [217]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   date                                                 973 non-null    datetime64[ns]
 1   dateYear                                             973 non-null    int64         
 2   dateMonth                                            973 non-null    int64         
 3   dateDay                                              973 non-null    int64         
 4   dateDayOfYear                                        973 non-null    int64         
 5   dateWeekOfMonth                                      973 non-null    int64         
 6   dateQuarter                                          973 non-null    int64         
 7   dateIsWeekend                                        973 non-null    int64         
 8   

In [218]:
for col in df1.columns:
    print(col, df1[col].nunique())

date 973
dateYear 3
dateMonth 12
dateDay 31
dateDayOfYear 365
dateWeekOfMonth 6
dateQuarter 4
dateIsWeekend 2
dateIsHolidayInd 2
dateWeekDayStartsMonday 7
dateWeekOfYearStartsMonday 53
firstDayWorkBefore 674
firstDayWorkAfter 673
firstDayNoHolidayNoSaturdayAfter 801
firstDayNoHolidayNoSundayAfter 809
firstDayNoSaturdayNoHolidayOrSundayHolidayIsHoliday 2
firstDayNoSaturdayNoHolidayOrSundayHoliday 813
isThuHolyday 2
isFriHolyday 2
isSatHolyday 2
isWeekendParcelServiceBreakInd 2
courierBillingPeriodStartDateFk 64
courierBillingPeriodEndDateFk 64


In [219]:
df1["isWeekendParcelServiceBreakInd"] = df1["isWeekendParcelServiceBreakInd"].astype(int)
df1["firstDayWorkBefore"] = pd.to_datetime(df1["firstDayWorkBefore"].astype(str), format='%Y%m%d')
df1["firstDayWorkAfter"] = pd.to_datetime(df1["firstDayWorkAfter"].astype(str), format='%Y%m%d')
df1["firstDayNoHolidayNoSaturdayAfter"] = pd.to_datetime(df1["firstDayNoHolidayNoSaturdayAfter"].astype(str), format='%Y%m%d')
df1["firstDayNoHolidayNoSundayAfter"] = pd.to_datetime(df1["firstDayNoHolidayNoSundayAfter"].astype(str), format='%Y%m%d')
df1["firstDayNoSaturdayNoHolidayOrSundayHoliday"] = pd.to_datetime(df1["firstDayNoSaturdayNoHolidayOrSundayHoliday"].astype(str), format='%Y%m%d')
df1["courierBillingPeriodStartDateFk"] = pd.to_datetime(df1["courierBillingPeriodStartDateFk"].astype(str), format='%Y%m%d')
df1["courierBillingPeriodEndDateFk"] = pd.to_datetime(df1["courierBillingPeriodEndDateFk"].astype(str), format='%Y%m%d')

In [220]:
#datetime_extract = ["firstDayWorkBefore", "firstDayWorkAfter", "firstDayNoHolidayNoSaturdayAfter", "firstDayNoHolidayNoSundayAfter",
#                    "firstDayNoSaturdayNoHolidayOrSundayHoliday", "courierBillingPeriodStartDateFk", "courierBillingPeriodEndDateFk"]
#for col in datetime_extract:
#    df1[col + "_year"] = df1[col].dt.year
#    df1[col + "_month"] = df1[col].dt.month
#    df1[col + "_day"] = df1[col].dt.day

#df1 = df1.drop(datetime_extract, axis = 1)

In [221]:
df1.iloc[-10:, 0:20]

,date,dateYear,dateMonth,dateDay,dateDayOfYear,dateWeekOfMonth,dateQuarter,dateIsWeekend,dateIsHolidayInd,dateWeekDayStartsMonday,dateWeekOfYearStartsMonday,firstDayWorkBefore,firstDayWorkAfter,firstDayNoHolidayNoSaturdayAfter,firstDayNoHolidayNoSundayAfter,firstDayNoSaturdayNoHolidayOrSundayHolidayIsHoliday,firstDayNoSaturdayNoHolidayOrSundayHoliday,isThuHolyday,isFriHolyday,isSatHolyday
963,2023-08-22,2023,8,22,234,4,3,0,0,2,35,2023-08-21,2023-08-23,2023-08-23,2023-08-23,0,2023-08-23,0,0,0
964,2023-08-23,2023,8,23,235,4,3,0,0,3,35,2023-08-22,2023-08-24,2023-08-24,2023-08-24,0,2023-08-24,0,0,0
965,2023-08-24,2023,8,24,236,4,3,0,0,4,35,2023-08-23,2023-08-25,2023-08-25,2023-08-25,0,2023-08-25,0,0,0
966,2023-08-25,2023,8,25,237,4,3,0,0,5,35,2023-08-24,2023-08-28,2023-08-27,2023-08-26,0,2023-08-27,0,0,0
967,2023-08-26,2023,8,26,238,4,3,1,0,6,35,2023-08-25,2023-08-28,2023-08-27,2023-08-28,0,2023-08-27,0,0,0
968,2023-08-27,2023,8,27,239,5,3,1,0,7,35,2023-08-25,2023-08-28,2023-08-28,2023-08-28,0,2023-08-28,0,0,0
969,2023-08-28,2023,8,28,240,5,3,0,0,1,36,2023-08-25,2023-08-29,2023-08-29,2023-08-29,0,2023-08-29,0,0,0
970,2023-08-29,2023,8,29,241,5,3,0,0,2,36,2023-08-28,2023-08-30,2023-08-30,2023-08-30,0,2023-08-30,0,0,0
971,2023-08-30,2023,8,30,242,5,3,0,0,3,36,2023-08-29,2023-08-31,2023-08-31,2023-08-31,0,2023-08-31,0,0,0
972,2023-08-31,2023,8,31,243,5,3,0,0,4,36,2023-08-30,2023-09-01,2023-09-01,2023-09-01,0,2023-09-01,0,0,0


In [ ]:
df1.iloc[-10:, 20:]

,isWeekendParcelServiceBreakInd,courierBillingPeriodStartDateFk,courierBillingPeriodEndDateFk
963,0,2023-08-16,2023-08-31
964,0,2023-08-16,2023-08-31
965,0,2023-08-16,2023-08-31
966,0,2023-08-16,2023-08-31
967,0,2023-08-16,2023-08-31
968,0,2023-08-16,2023-08-31
969,0,2023-08-16,2023-08-31
970,0,2023-08-16,2023-08-31
971,0,2023-08-16,2023-08-31
972,0,2023-08-16,2023-08-31


In [223]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   date                                                 973 non-null    datetime64[ns]
 1   dateYear                                             973 non-null    int64         
 2   dateMonth                                            973 non-null    int64         
 3   dateDay                                              973 non-null    int64         
 4   dateDayOfYear                                        973 non-null    int64         
 5   dateWeekOfMonth                                      973 non-null    int64         
 6   dateQuarter                                          973 non-null    int64         
 7   dateIsWeekend                                        973 non-null    int64         
 8   